Wczytanie bibliotek:

In [1]:
import numpy as np
import pandas as pd
import random 
from random import randint
import matplotlib.pyplot as plt

Wczytanie zbioru danych:

In [2]:
df = pd.read_csv("players_22.csv", low_memory=False)
df.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


Zdefiniowanie generowania sie pojedynczego skladu pilkarzy:

In [3]:
#Najważniejsze atrybuty danego piłkarza potrzebne w analizie:
class Player:
    def __init__(self, name, overall, salary, position):
        self.name = name
        self.overall = overall
        self.salary = salary
        self.position = position

In [4]:
#Generowanie gracza na daną pozycję tzn. poprzez zdefiniowanie zbioru player_df jako zbioru macierzystego, 
#ale z zawodnikami tylko dla danej pozycji:
def generate_player(player_df, genes):
    gen_df = player_df.sort_values(['overall'], ascending=False)
    player = random.randint(0, len(gen_df)-1)
    player_item = Player(
        gen_df.iloc[player]['short_name'],
        gen_df.iloc[player]['overall'],
        gen_df.iloc[player]['wage_eur'],
        gen_df.iloc[player]['club_position']
    )
    return player_item

In [5]:
#Generowanie indywidualnego składu piłkarzy poprzezy wybór pojeynczych graczy na daną pozycję poprzez użycie funkcji
#generate_player (wybór 2 dla pozycji CB):
def create_individual(df):
    genes = list()
    positions = ['GK','LB','CB','RB','CDM','CM','CAM','LW','ST','RW']
    for pos in positions:
        player_df = df[df['club_position'] == pos]
        for i in range(1):
            player_item = generate_player(player_df, genes)
            genes.append(player_item)
        if pos == 'CB':
            for i in range(1):
                player_df=player_df[player_df['short_name'] != player_item.name]
                player_item = generate_player(player_df, genes)
                genes.append(player_item)

    return genes

Liczenie score'u danego zespołu:

In [6]:
#Zdefiniowanie funkcji score do obliczania score dla danego wylosowanego zespołu - score jako suma overall zawodników,
#dla pensji większych od maksymalnego budżetu score=0.
def scoring(budget, individual):
    score=0
    salary=0
    for player in individual:
        score += player.overall
        salary += player.salary

    if float(budget-salary)<0:
        score = 0
        
    return float(score)

Inicjalna pula składów

In [7]:
#Funkcja losująca inicjalną pulę rodzicielską w postaci num_parents składów zawodników, z default'u mamy w inicjalnej puli
#rodzicielskiej 100 składów.
def selection(df, initial_pop=100):
    init_pop=[]
    for i in range(initial_pop):
        init_pop.append(create_individual(df))
    return init_pop

Operator selekcji - roullette wheel selection

In [8]:
#Wybór losowego składu z danej puli składów z prawdopodbieństwem równym score dla danego składu podzielić przez score dla
#wszystkich składów z puli
def roulette_select(parent, budget, num):

    fitness_list=[]
    for i in range (len(parent)):
        fitness_list.append(scoring(budget, parent[i]))
        
    total_fit = sum(fitness_list)
    relative_fitness = [f/total_fit for f in fitness_list]
    
    parents = random.choices(parent, weights = relative_fitness, k = num)
    return parents

Operator krzyżowania - one-point crossover

In [9]:
#Zastosowany operator krzyżowania to one-point crossover. W zdefiniowanej fukcji można wybrać sobie crossover_point, który 
#default'owo ustawiony jest na 5, tzn. krzyżowanie przebiega w ten sposób, że jeden offspring przyjmuje skład z 
#5 zawodnikami od pierwszego parent'a (pozycje: GK, LB, CB, CB, RB) oraz 6 zawodników następnych od drugiego (pozostałe
#pozycje), a drugi offspring odwrotnie przyjmuje 5 od parenta'a drugiego (pozycje: GK, LB, CB, CB, RB) oraz 6 od pierwszego
#(pozostałe pozycje). W zaimplementowanym krzyżowaniu nie ma problemu, że przez krzyżowanie powstaje skład z np. 
#dwoma bramkarzami.
def crossover(parents, crossover_point=5):
    i=0
    num_offsprings=0
    offsprings = [0]*len(parents)
    while (num_offsprings < len(parents)):
        try:
            parent1_index = parents[i]
            parent2_index = parents[i+1]
            offsprings[i]=parent1_index[0:crossover_point]
            offsprings[i][crossover_point:]=(parent2_index[crossover_point:])
            offsprings[i+1]=parent1_index[crossover_point:]
            offsprings[i+1][crossover_point:]=(parent2_index[0:crossover_point])
        except:
            offsprings[i]=parent1_index
        i=i+2
        num_offsprings=num_offsprings+2
    return offsprings

Operator mutacji

In [10]:
#Zaimplementowany operator mutacji zmienia z danym mutation_rate pojedynczego zawodnika z danego składu z puli dzieci
#offspring na innego zawodnika o tej samej pozycji ze zbioru wszystkich zawodników. Dzięki zmianie na zawodnika o tej 
#pozycji nie ma problemu z tym, że w danym składzie mamy np. dwóch bramkarzy. W zdefiniowanej funkcji można wybrać 
#mutation_rate, default'owo to 0.15.
def mutate(offsprings, df, mutation_rate=0.15):
    new_list= []
    new = []
    for i in range(len(offsprings)):
        individual=offsprings[i]
        for j in range(len(individual)):
            player_df = df[df['club_position'] == individual[j].position]
            if mutation_rate > random.random():
                for k in range(1):
                    player_item = generate_player(player_df, df)
                new.append(player_item)
            else:
                new.append(individual[j])
        new_list.append(new)
        new=[]
    return new_list

Warunek ograniczający - budżet na pensje

In [39]:
budget=500000
num_parent=200

Inicjalna iteracja algorytmu GA (pierwsza iteracja)

In [40]:
#Inicjalna iteracja algorytmu GA. W pierwszym kroku losujemy daną pulę num_parent składów (100 składów), a następnie 
#przeprowadzamy kolejno algorytm selekcji, krzyżowania i mutacji. 
start=create_individual(df)
init_pop = selection(df,num_parent)
parents=roulette_select(init_pop, budget, num_parent)
offsprings=crossover(parents)
mutations=mutate(offsprings, df)
best_score=0
best_ind=start
for i in (range(len(mutations))):
    score = scoring(budget, mutations[i])
    if score>best_score:
        best_score=score
        best_ind=mutations[i]
    else:
        pass

Algorytm Genetic Algorithm

In [37]:
#Z puli składów powstałej w wyniku pierwszej iteracji algorytmu przeprowadzamy kolejno algorytm selekcji, krzyżowania 
#i mutacji, otrzymując kolejną pulę dla której znowu stosujemy kolejno powyższe operatory. Dla funkcji można podać liczbę
#iteracji default'owo ustawioną na 99. Funkcja na końcu zwraca najlepszy skład z puli po wykonaniu danej liczby iteracji.
def GA(df, iterations=99, budget=budget, mutations=mutations, best_score=best_score, best_ind=best_ind):
    for j in (range(iterations)):
        parents=roulette_select(mutations,budget,num_parent)
        offsprings=crossover(parents)
        mutations=mutate(offsprings, df)
        for i in (range(len(mutations))):
            score = scoring(budget, mutations[i])
            if score>best_score:
                best_score=score
                best_ind=mutations[i]
        print(best_score)
        j+=1
    return best_ind

Wynik algorytmu

In [41]:
GA(df, 14)

809.0
809.0
810.0
810.0
810.0
810.0
810.0
810.0
810.0
810.0
810.0
810.0
810.0
810.0


In [42]:
for player in best_ind:
    print(player.name, player.position, player.overall)

Munir GK 73
M. Bard LB 72
I. Traoré CB 75
L. Koscielny CB 78
N. Boushal RB 62
M. Brozović CDM 84
D. Kohr CM 75
A. Hjulsager CAM 73
J. Quiñones LW 68
Matheus Bardeira ST 78
Santi da Bruma RW 72
